# Sentence Classfication using RNN

In [1]:
import csv
import time
import spacy
import torch
import random
from torch import nn
from torchtext.legacy import data
from torchtext.datasets import AG_NEWS
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator, GloVe
from string import punctuation

tokenizer = get_tokenizer('basic_english')
# train_iter, test_iter = AG_NEWS()

# with open('./data/train_ag_news.csv', mode='w') as f:
#     fieldnames = ['sentence', 'label']
#     csv_writer = csv.writer(f)
#     csv_writer.writerow(fieldnames)
#     for label, sentence in train_iter:
#         csv_writer.writerow([sentence, label])
        
# with open('./data/test_ag_news.csv', mode='w') as f:
#     fieldnames = ['sentence', 'label']
#     csv_writer = csv.writer(f)
#     csv_writer.writerow(fieldnames)
#     for label, sentence in test_iter:
#         csv_writer.writerow([sentence, label])

In [ ]:
SEED = 456

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [2]:
SEQ_LEN = 45
BATCH_SIZE = 128
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
spacy_tokenizer = get_tokenizer('spacy', language='en_core_web_sm')

def preprocessor(sentence):
    temp = [''.join([l for l in word if l not in punctuation]) for word in sentence]
    return [word for word in temp if word not in ['', ' ']]

SENTENCE = data.Field(
    tokenize  = spacy_tokenizer,
    preprocessing = preprocessor,
    lower       = True,
    batch_first = True,
    fix_length  = SEQ_LEN
)
LABEL = data.LabelField(sequential = False, batch_first=True)
        
fields = [('sentence', SENTENCE), ('label', LABEL)]
training_data = data.TabularDataset(
    path='train_ag_news.csv',
    format='csv',
    fields=fields,
    skip_header=True,
)

test_data = data.TabularDataset(
    path='test_ag_news.csv',
    format='csv',
    fields=fields,
    skip_header=True,
)

train_data, val_data = training_data.split(split_ratio=0.8, random_state=random.seed(SEED))
SENTENCE.build_vocab(train_data, vectors=GloVe(name = "42B", dim=300), max_size=10000, min_freq=1)
LABEL.build_vocab(train_data)

train_dataloader, val_dataloader, test_dataloader = data.BucketIterator.splits(
    (train_data, val_data, test_data),
    batch_size=BATCH_SIZE,
    device=device,
    shuffle=True,
    sort = False,
)

In [3]:
class BiRNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_hiddens, num_layers, num_labels, **kwargs):
        super(BiRNN, self).__init__(**kwargs)
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, num_hiddens, 
                            num_layers=num_layers, 
                            batch_first=True, 
                            bidirectional=True)
        self.fc = nn.Linear(4 * num_hiddens, num_labels)

    def forward(self, inputs):
        embeddings = self.embedding(inputs)
#         self.lstm.flatten_parameters()
        outputs, (h_state, c_state) = self.lstm(embeddings)
        return self.fc(torch.cat((outputs[:,0,:], outputs[:,-1,:]),dim=1))

In [4]:
def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 100
    start_time = time.time()

    for batch_idx, batch in enumerate(dataloader):
        optimizer.zero_grad()
        sentence, label = batch.sentence, batch.label
        predicted_label = model(sentence)
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.shape[0]
        if batch_idx % log_interval == 0 and batch_idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, batch_idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for batch_idx, batch in enumerate(dataloader):
            sentence, label = batch.sentence, batch.label
            predicted_label = model(sentence)
#             loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.shape[0]
    return total_acc/total_count

In [6]:
VOCAB_SIZE = len(SENTENCE.vocab)
EMBED_DIM = SENTENCE.vocab.vectors.shape[1]
NUM_HIDDENS = 64
NUM_LABELS = 4
NUM_LAYERS = 4 
EPOCHS = 10 
LR = 5  

model = BiRNN(VOCAB_SIZE, EMBED_DIM, NUM_HIDDENS, NUM_LAYERS, NUM_LABELS)
model.embedding.weight.data = SENTENCE.vocab.vectors
model.to(device)


criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None


for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(val_dataloader)
    if total_accu is not None and total_accu > accu_val:
      scheduler.step()
    else:
       total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

| epoch   1 |   100/  750 batches | accuracy    0.433
| epoch   1 |   200/  750 batches | accuracy    0.695
| epoch   1 |   300/  750 batches | accuracy    0.797
| epoch   1 |   400/  750 batches | accuracy    0.845
| epoch   1 |   500/  750 batches | accuracy    0.869
| epoch   1 |   600/  750 batches | accuracy    0.883
| epoch   1 |   700/  750 batches | accuracy    0.894
-----------------------------------------------------------
| end of epoch   1 | time: 37.26s | valid accuracy    0.888 
-----------------------------------------------------------
| epoch   2 |   100/  750 batches | accuracy    0.899
| epoch   2 |   200/  750 batches | accuracy    0.898
| epoch   2 |   300/  750 batches | accuracy    0.906
| epoch   2 |   400/  750 batches | accuracy    0.903
| epoch   2 |   500/  750 batches | accuracy    0.903
| epoch   2 |   600/  750 batches | accuracy    0.905
| epoch   2 |   700/  750 batches | accuracy    0.911
-----------------------------------------------------------
| e

In [7]:
accu_test = evaluate(test_dataloader)
print('test accuracy {:8.3f}'.format(accu_test))

test accuracy    0.926
